In [9]:
import numpy as np
import pandas as pd
import rasterio
from scipy import stats
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
from matplotlib.colors import BoundaryNorm
from cartopy.feature import NaturalEarthFeature
import cartopy.crs as ccrs
import cartopy

In [10]:
lat = np.array([49.9166666666664 - i * 0.0416666666667 for i in range(357)])
lon = np.array([-105.0416666666507 + i * 0.0416666666667 for i in range(722)])
Lon, Lat = np.meshgrid(lon, lat)

am = rasterio.open('PRISM_tmin_stable_4kmD2_19810101_bil.bil')
a = am.read()[0, :357, 479:1201]
mask = np.where(a>-1000, 1, 0)

tmin81 = np.load('./prism/mw_tmin/1981_tmin.npz')['tmin']
t, y, x = tmin81.shape
y, x

(357, 722)

In [ ]:
# MonStart = [59, 90, 120]
# MonStartLeap = [60, 91, 121]
# MonDays = [31, 30, 31]
# Tcriteria = [0, -5, -10, -15, -20]
# March = np.zeros((38, 5, 31))
# April = np.zeros((38, 5, 30))
# May = np.zeros((38, 5, 31))
# count = [March, April, May]

# for yl in range(38):
#     tmin = np.load(f'./prism/mw_tmin/{1981+yl}_tmin.npz')['tmin']
#     tmin = np.where(tmin>-100, tmin, np.nan)
#     if (1981+yl)%4:
#         MonS = MonStart
#     else:
#         MonS = MonStartLeap
#     for mi in range(3):
#         for ti in range(5):
#             count[mi][yl, ti, :] = np.array([np.where(tmin[MonS[mi]+di, :, :]<Tcriteria[ti], 1, 0).sum()/(y*x) for di in range(MonDays[mi])])

# np.save('./var/MarchCount', count[0])
# np.save('./var/AprilCount', count[1])
# np.save('./var/MayCount', count[2])

In [ ]:
# states = ['MI', 'ND', 'SD', 'NE', 'KS', 'OK', 'MN', 'IA', 'MO', 'WI', 'IL', 'IN', 'OH', 'KY', 'WV', 'PA']
# mdmask = [np.load(f'./var/{states[ii]}_mask.npy') for ii in range(16)]

# MonStart = [59, 90, 120]
# MonStartLeap = [60, 91, 121]
# MonDays = [31, 30, 31]
# Tcriteria = [0, -5, -10, -15, -20]
# March = np.zeros((38, 5, 16, 31))
# April = np.zeros((38, 5, 16, 30))
# May = np.zeros((38, 5, 16, 31))
# count = [March, April, May]

# for yl in range(38):
#     tmin = np.load(f'./prism/mw_tmin/{1981+yl}_tmin.npz')['tmin']
#     tmin = np.where(tmin>-100, tmin, np.nan)
#     if (1981+yl)%4:
#         MonS = MonStart
#     else:
#         MonS = MonStartLeap
#     for mi in range(3):
#         for ti in range(5):
#             for si in range(16):
#                 count[mi][yl, ti, si, :] = np.array([np.where(np.where(mdmask[si], tmin[MonS[mi]+di, :, :], np.nan)<Tcriteria[ti], 1, 0).sum() / mdmask[si].sum() for di in range(MonDays[mi])])

# np.save('./var/MarchCountState', count[0])
# np.save('./var/AprilCountState', count[1])
# np.save('./var/MayCountState', count[2])

In [ ]:
# March = np.load('./var/MarchCount.npy') * (y*x) / mask.sum()
# April = np.load('./var/AprilCount.npy') * (y*x) / mask.sum()
# May = np.load('./var/MayCount.npy') * (y*x) / mask.sum()

In [ ]:
# fig = plt.figure(figsize=(8, 6))
# X = np.arange(1981, 2019)
# plt.plot(X, March[:, 0, :].sum(axis=1), 'b-o', lw=1, ms=3)
# plt.plot(X, March[:, 1, :].sum(axis=1), 'g-o', lw=1, ms=3)
# plt.plot(X, March[:, 2, :].sum(axis=1), 'y-o', lw=1, ms=3)
# plt.plot(X, March[:, 3, :].sum(axis=1), 'm-o', lw=1, ms=3)
# plt.plot(X, March[:, 4, :].sum(axis=1), 'c-o', lw=1, ms=3)
# plt.xticks(np.arange(1981, 2018, 3), [str(X[i]) for i in range(0, 38, 3)])
# plt.ylim(0, 25)
# plt.legend(['Tmin<0\u2103', 'Tmin<-5\u2103', 'Tmin<-10\u2103', 'Tmin<-15\u2103', 'Tmin<-20\u2103'], frameon=False)
# plt.title('31 days Sum of Area Percentage in March', fontsize='large', fontweight='bold')
# plt.savefig('./plot/Composite/MarchTotal.png', bbox_inches='tight')

<font size=4 color=blue>slope of area percentage

In [18]:
def color_significance_red(s):
    if s[-1] == '*':
        color = 'red'
    else:
        color = 'black'
    return 'color: %s' % color

In [22]:
states = ['MI', 'ND', 'SD', 'NE', 'KS', 'OK', 'MN', 'IA', 'MO', 'WI', 'IL', 'IN', 'OH', 'KY', 'WV', 'PA']
mdmask = [np.load(f'./var/{states[ii]}_mask.npy') for ii in range(16)]
Tcriteria = [0, -5, -10, -15, -20]

March = np.load('./var/MarchCountState.npy').sum(axis=3)
April = np.load('./var/AprilCountState.npy').sum(axis=3)
May = np.load('./var/MayCountState.npy').sum(axis=3)
data = [March, April, May]
slope = np.zeros((3, 5, 16))
pvalue = np.zeros((3, 5, 16))
X = np.linspace(1, 38, 38)
for mi in range(3):
    for ti in range(5):
        for si in range(16):
            r = stats.linregress(X, data[mi][:, ti, si])
            slope[mi, ti, si] = r.slope
            pvalue[mi, ti, si] = r.pvalue
table = [[[] for i in range(5)] for j in range(16)]
mi = 1
for si in range(16):
    for ti in range(5):
        if pvalue[mi, ti, si] < .1:
            table[si][ti] = f'{(slope[mi, ti, si]):.3f}*'
        else:
            table[si][ti] = f'{(slope[mi, ti, si]):.3f}'
table = pd.DataFrame(table)
table = table.rename(index={i:states[i] for i in range(16)}, columns={i:Tcriteria[i] for i in range(5)})
table.style.applymap(color_significance_red)

<font size=4 color=blue>Freezing days count

In [ ]:
MonStart = [59, 90, 120]
MonStartLeap = [60, 91, 121]
MonDays = [31, 30, 31]
Tcriteria = [0, -5, -10, -15, -20]
count = np.zeros((38, 3, y, x))
for yl in range(38):
    tmin = np.load(f'./prism/mw_tmin/{1981+yl}_tmin.npz')['tmin']
    tmin = np.where(tmin>-100, tmin, np.nan)
    if (1981+yl)%4:
        MonS = MonStart
    else:
        MonS = MonStartLeap
    for mi in range(3):
        S = MonS[mi]
        E = S + MonDays[mi]
        for ti in range(1):
            count[yl, mi, :, :] = np.where(tmin[S:E, :, :]<Tcriteria[ti], 1, 0).sum(axis=0)

np.save('./var/Min0DaysCount', count)

In [4]:
count = np.load('./var/Min0DaysCount.npy')
count_mean = count.mean(axis=0)
count_mean = np.where(abs(count_mean)<.000001, np.nan, count_mean)
count_std = count.std(axis=0)
count_std = np.where(abs(count_std)<.000001, np.nan, count_std)
count = np.where(count==0, np.nan, count)
# slope = np.zeros((3, y, x)) * np.nan
# pvalue = np.zeros((3, y, x)) * np.nan
# X = np.linspace(1, 38, 38)

# for i in range(y):
#     for j in range(x):
#         if mask[i, j]:
#             for mi in range(3):
#                 counttemp = count[:, mi, i, j]
#                 flag = np.where(np.isnan(counttemp), False, True)
#                 if flag.sum()>1:
#                     r = stats.linregress(X[flag], counttemp[flag])
#                     slope[mi, i, j] = r.slope
#                     pvalue[mi, i, j] = r.pvalue
# np.save('./var/Min0DaysSlope', slope)
# np.save('./var/Min0DaysPvalue', pvalue)
slope = np.load('./var/Min0DaysSlope.npy')
pvalue = np.load('./var/Min0DaysPvalue.npy')

In [22]:
png = ['Min0DaysCount_mean', 'Min0DaysCount_std', 'Min0DaysCount_trend']
title = ['38 year mean', 'Standard Deviation', 'Trend']
month = ['March', 'April', 'May']
data = [count_mean, count_std/count_mean, slope]
extent = [-105, -75, 34, 49]
ll = [0, 0, -.3]
lr = [30, 2, .3]
step = [5, .4, .1]
color = ['jet_r', 'jet', 'bwr']
extend = ['max', 'max', 'both']
ch = [.625, .325, .025]
fig, axs = plt.subplots(3, 3, figsize=(15, 9))
for ii in range(3):
    for mi in range(3):
        ax = plt.subplot(3,3,ii*3+mi+1, projection=ccrs.AlbersEqualArea(np.mean(extent[:2]), np.mean(extent[2:])))
        ax.set_extent(extent)
        ax.add_feature(cartopy.feature.STATES.with_scale('10m'), lw=.5)
        ax.add_feature(cartopy.feature.BORDERS, lw=.3, linestyle=':')
        ax.add_feature(cartopy.feature.COASTLINE, lw=.3, linestyle=':')
        ax.add_feature(cartopy.feature.LAKES, edgecolor='black', facecolor='white', lw=.3)

        levels = MaxNLocator(nbins=100).tick_values(ll[ii], lr[ii])
        cmap = plt.get_cmap(color[ii])
        norm = BoundaryNorm(levels, ncolors=cmap.N, clip=True)

        pc = plt.pcolormesh(Lon, Lat, np.where(mask, data[ii][mi, :, :], np.nan), norm=norm, cmap=cmap, transform=ccrs.PlateCarree())
        if ii == 2:
            plt.scatter(Lon, Lat, np.where(np.logical_and(pvalue[mi, :, :]<0.05, mask), 1, np.nan), 'grey', alpha=.1, transform=ccrs.PlateCarree())

        plt.title(f'{title[ii]} in {month[mi]}', fontsize=13, fontweight='bold')
        fig.subplots_adjust(bottom=0, top=0.9, left=0.1, right=0.9, wspace=0.05, hspace=0.02)
        
        if mi == 2:
            cb_ax = fig.add_axes([0.905, ch[ii], 0.01, 0.25])
            cbar = plt.colorbar(pc, cax=cb_ax, ticks=np.arange(ll[ii], lr[ii]+step[ii], step[ii]), extend=extend[ii])
plt.suptitle('Freezing Days (Tmin<0\u2103) from 1981 to 2018', fontsize=14, fontweight='bold')
plt.savefig(f'./plot/Composite/FreezingDaysMeanStdTrend.png', bbox_inches='tight')
plt.close()

<ipython-input-22-7f6c62bd0553>:28: RuntimeWarning: invalid value encountered in less
  plt.scatter(Lon, Lat, np.where(np.logical_and(pvalue[mi, :, :]<0.05, mask), 1, np.nan), 'grey', alpha=.1, transform=ccrs.PlateCarree())


<font size=4 color=blue>trend of freezing days in April at individual location

In [5]:
latloc = [45.993005, 44.661906, 41.309247, 39.710815, 36.289078]
lonloc = [-87.647226, -84.546377, -87.292439, -81.960948, -97.326834]
yloc = np.zeros(5)
xloc = np.zeros(5)
for li in range(5):
    yloc[li] = np.argmin(abs(lat-latloc[li]))
    xloc[li] = np.argmin(abs(lon-lonloc[li]))
yloc = yloc.astype(int)
xloc = xloc.astype(int)
Aprcount = count[:, 1, :, :]
X = np.linspace(1, 38, 38)
li = 4
# plt.figure(figsize=(8, 6))
# plt.bar(X, Aprcount[:, yloc[li], xloc[li]])
# plt.xticks(np.arange(1, 39, 2), [str(1981+i)[2:] for i in range(0, 39, 2)])
# plt.ylim(0, 31)
# plt.title(str(li+1), fontsize='large', fontweight='bold')
# plt.show()

In [8]:
# fig = plt.figure(figsize=(8, 6))
# extent = [-105, -75, 34, 49]
# ax = plt.axes(projection=ccrs.AlbersEqualArea(np.mean(extent[:2]), np.mean(extent[2:])))
# ax.set_extent(extent)
# states = NaturalEarthFeature(category="cultural", scale="50m",
#                              facecolor="none",
#                              name="admin_1_states_provinces_shp")
# ax.add_feature(states, linewidth=.3, edgecolor="black")
# ax.add_feature(cartopy.feature.BORDERS, lw=.3, linestyle=':')
# ax.add_feature(cartopy.feature.COASTLINE, lw=.3, linestyle=':')
# ax.add_feature(cartopy.feature.LAKES, edgecolor='black', facecolor='white', lw=.3)

# levels = MaxNLocator(nbins=100).tick_values(-.3, .3)
# cmap = plt.get_cmap('bwr')
# norm = BoundaryNorm(levels, ncolors=cmap.N, clip=True)

# pc = plt.pcolormesh(Lon, Lat, slope[1, :, :], cmap=cmap, norm=norm, transform=ccrs.PlateCarree())
# plt.scatter(Lon, Lat, np.where(pvalue[1, :, :]<0.05, 1, np.nan), 'grey', alpha=.1, transform=ccrs.PlateCarree())
# for k in range(5):
#     ax.plot(lonloc[k], latloc[k], 'c*', markersize=7, transform=ccrs.PlateCarree())
#     ax.text(lonloc[k]-.5, latloc[k]+.3, f'{k+1}', c='m', fontsize=15, transform=ccrs.PlateCarree())

# plt.title(f'Trend of Freezing Days (Tmin<0\u2103) from 1981 to 2018', fontsize=13, fontweight='bold')

# fig.subplots_adjust(bottom=0, top=0.95, left=0.05, right=0.9)
# cb_ax = fig.add_axes([0.905, 0.12, 0.02, 0.72])
# cbar = fig.colorbar(pc, cax=cb_ax, ticks=np.arange(-.3, .31, .1), extend='both') 

In [6]:
# count = np.load('./var/Min0DaysCount.npy')
# count_mean = count.sum(axis=1).mean(axis=0)
# count_std = count.sum(axis=1).std(axis=0)
# slope = np.zeros((y, x))
# pvalue = np.zeros((y, x))
# X = np.linspace(1, 38, 38)

# for i in range(y):
#     for j in range(x):
#         if mask[i, j]:
#             r = stats.linregress(X, count[:, :, i, j].sum(axis=1))
#             slope[i, j] = r.slope
#             pvalue[i, j] = r.pvalue

title = ['38 year mean', 'Standard Deviation', 'Trend']
data = [count_mean, count_std, slope]
extent = [-105, -75, 34, 49]
ll = [0, 0, -.3]
lr = [60, 12, .3]
step = [10, 2, .1]
color = ['jet_r', 'jet', 'bwr']
extend = ['max', 'max', 'both']
ch = [.625, .325, .025]
fig, axs = plt.subplots(3, 1, figsize=(5, 9.5))
for ii in range(3):
    ax = plt.subplot(3,1,ii+1, projection=ccrs.AlbersEqualArea(np.mean(extent[:2]), np.mean(extent[2:])))
    ax.set_extent(extent)
    ax.add_feature(cartopy.feature.STATES.with_scale('10m'), lw=.5)
    ax.add_feature(cartopy.feature.BORDERS, lw=.3, linestyle=':')
    ax.add_feature(cartopy.feature.COASTLINE, lw=.3, linestyle=':')
    ax.add_feature(cartopy.feature.LAKES, edgecolor='black', facecolor='white', lw=.3)

    levels = MaxNLocator(nbins=100).tick_values(ll[ii], lr[ii])
    cmap = plt.get_cmap(color[ii])
    norm = BoundaryNorm(levels, ncolors=cmap.N, clip=True)

    pc = plt.pcolormesh(Lon, Lat, np.where(mask, data[ii], np.nan), norm=norm, cmap=cmap, transform=ccrs.PlateCarree())
    if ii == 2:
        plt.scatter(Lon, Lat, np.where(np.logical_and(pvalue<0.05, mask), 1, np.nan), 'grey', alpha=.1, transform=ccrs.PlateCarree())

    plt.title(f'{title[ii]}', fontsize=13, fontweight='bold')
    fig.subplots_adjust(bottom=0, top=0.9, left=0.1, right=0.9, wspace=0.05, hspace=0.02)

    cb_ax = fig.add_axes([0.91, ch[ii], 0.02, 0.25])
    cbar = plt.colorbar(pc, cax=cb_ax, ticks=np.arange(ll[ii], lr[ii]+step[ii], step[ii]), extend=extend[ii])
plt.suptitle('Freezing Days (Tmin<0\u2103) from 1981 to 2018', fontsize=14, fontweight='bold')
plt.savefig(f'./plot/Composite/FreezingDaysMeanStdTrend_Total.png', bbox_inches='tight')
plt.close()

<font size=4 color=blue>EOF on count above

In [3]:
from sklearn import preprocessing
scaler  = preprocessing.StandardScaler()
from sklearn.decomposition import PCA

In [19]:
t = 38

count = np.load('./var/Min0DaysCount.npy')
count_anomaly = count[:, 2, :, :] - count[:, 2, :, :].mean(axis=0)

X1 = count_anomaly.reshape((t, y*x))
X1 = np.ma.masked_array(X1, np.isnan(X1))

nonflag = X1.sum(0).mask
flag = ~nonflag
X1 = X1[:,flag]
X1 = (X1 - X1.mean()) / X1.std()

skpca = PCA()
pca1 = skpca.fit(X1)
X1_pc = pca1.transform(X1)

pc = (X1_pc[:, :2] - X1_pc[:, :2].mean()) / X1_pc[:, :2].std()
# count_anomaly.shape

In [ ]:
plt.figure(figsize=(9,6))
plt.plot(pca1.explained_variance_ratio_[0:10]*100, 'b-o')
plt.xticks(np.arange(10))
plt.grid()
plt.title('The percentage of the total variance explained by the first ten EOF modes', fontweight='bold')
pca1.explained_variance_ratio_[0:10]

In [8]:
n_comp = 1
EOFs = pca1.components_[n_comp,:]
EOF_recons = np.ones(x*y) * -999.
EOF_recons[flag] = EOFs[:]
EOF_recons = np.ma.masked_values(EOF_recons.reshape((y, x)), -999.)

title = ['EOF1 59.81%', 'EOF2 9.91%']
extent = [-105, -75, 34, 49]
ii = 1
fig = plt.figure(figsize=(8, 6))
ax = plt.axes(projection=ccrs.AlbersEqualArea(np.mean(extent[:2]), np.mean(extent[2:])))
ax.set_extent(extent)
ax.add_feature(cartopy.feature.STATES.with_scale('10m'), lw=.5)
ax.add_feature(cartopy.feature.BORDERS, lw=.3, linestyle=':')
ax.add_feature(cartopy.feature.COASTLINE, lw=.3, linestyle=':')
ax.add_feature(cartopy.feature.LAKES, edgecolor='black', facecolor='white', lw=.3)

levels = MaxNLocator(nbins=100).tick_values(-1, 1)
cmap = plt.get_cmap('bwr')
norm = BoundaryNorm(levels, ncolors=cmap.N, clip=True)

pc = plt.pcolormesh(Lon, Lat, np.where(mask, 100*EOF_recons, np.nan), cmap=cmap, norm=norm, transform=ccrs.PlateCarree())
plt.title(f'{title[ii]}', fontsize=13, fontweight='bold')
fig.subplots_adjust(bottom=0, top=0.95, left=0.05, right=0.9)
cb_ax = fig.add_axes([0.905, 0.12, 0.02, 0.72])
cbar = fig.colorbar(pc, cax=cb_ax, ticks=np.arange(-1, 1.1, .2), extend='both')

In [ ]:
ii = 0
pc = (X1_pc[:, :2] - X1_pc[:, :2].mean()) / X1_pc[:, :2].std()
pc = pc[:, ii]
plt.figure(figsize=(9,6))
plt.plot(pc)
plt.xlabel('Time from 1981 to 2018', fontweight='bold')
plt.ylabel('Normalized Index', fontweight='bold')
plt.title(f'PC{ii+1}', fontsize='large', fontweight='bold')
plt.xticks(np.arange(0, 38, 4),[f'{1981+i}' for i in range(0, 38, 4)], fontweight='bold')
plt.yticks(np.arange(-4, 5), fontweight='bold')
plt.ylim(-4, 4)
plt.xlim(0, 38)
plt.hlines(0, 0, 38, 'k', ls='--')
plt.show()

<font size=4 color=blue>regression on climate variables

In [9]:
import pygrib

In [21]:
DIR = '/home/mmfire/study/2019-FWI-nasa-raw-cook/Python/ERA5_DATA/'
grb = pygrib.open(DIR+'geopotential_500hPa_1979.grib')

Lat = grb.message(1).latlons()[0]
Lon = grb.message(1).latlons()[1] - 180

extent = [-105, -75, 34, 49]
lonl = 300
lonr = 420
latr = 224
latl = 164
# np.argmin(abs(Lat[:,0]-extent[3]))

Lon = Lon[latl:latr, lonl:lonr]
Lat = Lat[latl:latr, lonl:lonr]
y, x = Lon.shape

In [11]:
#500 hPa
gph = np.zeros((38, y, x)) * np.nan
for yl in range(38):
    grb = pygrib.open(DIR+f'geopotential_500hPa_{1981+yl}.grib')
    temp = np.zeros((y, x))
    for mi in range(12):
        data = grb.message(mi+1).data()[0][latl:latr, lonl:lonr]
        temp = temp + data
    gph[yl, :, :] = temp / 12.
gph_anml = gph - gph.mean(axis=0)

In [55]:
#700 hPa
gph = np.zeros((38, y, x)) * np.nan
for yl in range(38):
    grb = pygrib.open(DIR+f'geopotential_700hPa_{1981+yl}.grib')
    temp = np.zeros((y, x))
    for mi in range(12):
        data = grb.message(mi+1).data()[0][latl:latr, lonl:lonr]
        temp = temp + data
    gph[yl, :, :] = temp / 12.
gph_anml = gph - gph.mean(axis=0)

In [56]:
slope = np.zeros((2, y, x)) * np.nan
pvalue = np.zeros((2, y, x)) * np.nan
for i in range(y):
    for j in range(x):
        for pi in range(2):
            r = stats.linregress(pc[:, pi], gph_anml[:, i, j])
            slope[pi, i, j] = r.slope
            pvalue[pi, i, j] = r.pvalue

In [64]:
# extent = [-105, -75, 34, 49]
# ii = 1
# fig = plt.figure(figsize=(8, 6))
# ax = plt.axes(projection=ccrs.AlbersEqualArea(np.mean(extent[:2]), np.mean(extent[2:])))
# ax.set_extent(extent)
# ax.add_feature(cartopy.feature.STATES.with_scale('10m'), lw=.5)
# ax.add_feature(cartopy.feature.BORDERS, lw=.3, linestyle=':')
# ax.add_feature(cartopy.feature.COASTLINE, lw=.3, linestyle=':')
# ax.add_feature(cartopy.feature.LAKES, edgecolor='black', facecolor='white', lw=.3)

# pcm = plt.contourf(Lon, Lat, slope[ii, :, :], cmap='bwr', levels=np.linspace(-60, 60, 100), transform=ccrs.PlateCarree(), extend='both')
# plt.scatter(Lon, Lat, np.where(pvalue[ii, :, :]<0.05, 1, np.nan), 'grey', alpha=.1, transform=ccrs.PlateCarree())
# plt.title('anomalous 700-hPa geopotential height (gpm) regressed on PC 2', fontsize=13, fontweight='bold')
# fig.subplots_adjust(bottom=0, top=0.95, left=0.05, right=0.9)
# cb_ax = fig.add_axes([0.905, 0.12, 0.02, 0.72])
# cbar = fig.colorbar(pcm, cax=cb_ax, ticks=np.arange(-60, 61, 12))